In [ ]:
import torch
import torch.optim as optim
import numpy as np
from transformer import Transformer
from metrics import Evaluator
from datasets import load_dataset
from utils import *
from training import *
from tokenizer import *

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device}")

In [ ]:
traindict = load_dataset("opus100", "de-en", split="train")
testdict = load_dataset("opus100", "de-en", split="test")

In [ ]:
train_inputs, train_labels = get_split(traindict, "en", "de", size=100)
test_inputs, test_labels = get_split(testdict, "en", "de", size=10)
trainset = Dataset(train_inputs, train_labels)
testset = Dataset(test_inputs, test_labels)

In [ ]:
trainframe = trainset.dataframe()
trainframe.head()

In [ ]:
testframe = testset.dataframe()
testframe.head()

In [ ]:
print(trainframe.isnull().values.any())
trainframe.describe()

In [ ]:
print(testframe.isnull().values.any())
testframe.describe()

In [ ]:
trainset.sample()

In [ ]:
testset.sample()

In [ ]:
corpus_en = trainset.corpus(split=0) + testset.corpus(split=0)
corpus_de = trainset.corpus(split=1) + testset.corpus(split=1)
tokenizer_en = Nerdimizer()
tokenizer_de = Nerdimizer()
tokenizer_en.train(corpus_en, size=50000)
tokenizer_de.train(corpus_de, size=50000)
translator = Translator(tokenizer_en, tokenizer_de)

In [ ]:
en_vocab, de_vocab = translator.vocab_size()
maxlen_train = trainset.maxlen(translator)
maxlen_test = testset.maxlen(translator)
maxlen = min(maxlen_train, maxlen_test, 200)
start, end, pad = tokenizer_en["[S]"], tokenizer_en["[E]"], tokenizer_en["[P]"]
print(f"Number of input tokens: {len(tokenizer_en)}\nNumber of output tokens: {len(tokenizer_de)}")

In [ ]:
translator.padon(maxlen, end=True, pad_id=pad)
translator.truncon(maxlen, end=True)
print(f"Maxlen: {maxlen}")

In [ ]:
tokenized_trainset = trainset.tokenized(translator, model=True)
dataloader = tokenized_trainset.dataloader(batch_size=32, drop_last=False)

In [ ]:
model = Transformer(en_vocab, de_vocab, maxlen, pad_id=pad, dm=256, nhead=8, layers=3, dff=1024)
optimizer = optim.Adam(model.parameters(), lr=1e-5, betas=(0.9, 0.98), eps=10e-9)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.9, patience=10)
evaluator = Evaluator(testset, translator, start, end, maxlen, sample=10, ngrams=4, threshold=20, 
                    mode="geometric", device=device)
clock = Clock()
checkpoint = Checkpoint(model, optimizer, scheduler, evaluator, clock, epochs=3, 
                    path="checkpoints/english-german", overwrite=False)
model.to(device);

In [ ]:
train(dataloader, model, optimizer, scheduler, evaluator, checkpoint, clock,
    epochs=5, warmups=100, verbose=True, device=device)

In [ ]:
checkpoint = Checkpoint(model, optimizer, scheduler)
checkpoint.load_checkpoint(path="checkpoints/english-german-3")

In [ ]:
retrain(dataloader, checkpoint, 11)